In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import pickle
import json
import functools
import matplotlib

import esipy as esi

from tqdm.notebook import tqdm, trange

import requests

In [2]:
import time

class Timer:    
    def __enter__(self):
        self.start = time.clock()
        return self

    def __exit__(self, *args):
        self.end = time.clock()
        self.interval = self.end - self.start
        
        print(self.interval)

with Timer() as t:
    time.sleep(1)

0.99985561326803


In [3]:
# Set up network access
esi_client = esi.EsiClient(
    transport_adapter = requests.adapters.HTTPAdapter(
        pool_connections=100,
        pool_maxsize=100,
        max_retries=10,
        pool_block=False
    )
)
esi_app    = esi.EsiApp()

app = esi_app.get_latest_swagger

W0222 20:35:06.531581 10488 client.py:81] Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
d:\programme\python 3.6.6\lib\site-packages\esipy\client.py:82: UserWarning: Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
  warnings.warn(warning_message)


In [4]:
def intd(f):
    return {int(k) : v for k,v in json.load(f).items()};

with open('universe/systems.txt') as f:
    systems = intd(f);

with open('universe/stargates.txt') as f:
    stargates = intd(f);

constellation_ids = esi_client.request(
    app.op['get_universe_constellations']()
).data

constellations = {
    i : esi_client.request(app.op['get_universe_constellations_constellation_id'](constellation_id = i)).data
    for i in tqdm(constellation_ids)
}

systems_graph = nx.Graph();

for system in systems:
    systems_graph.add_node(system);

for stargate in stargates.values():
    systems_graph.add_edge(stargate["system_id"], stargate["destination"]["system_id"])

root = 'Jita'
root_id = [v for v in systems if systems[v]["name"] == root][0]

# Limit to high-sec systems
subselect = [k for k in systems if systems[k]["security_status"] > 0.65]
subgraph = systems_graph.subgraph(subselect)

# Limit to component connected to Jita
subselect = nx.single_source_shortest_path_length(subgraph, root_id)
subgraph = systems_graph.subgraph(subselect)

landmark_names = ['Jita', 'Amarr', 'Rens', 'Hek', 'Dodixie', 'Oursulaert', 'Tash-Murkon Prime', 'Agil'];
landmarks = [v["system_id"] for v in systems.values() if v["name"] in landmark_names]

@functools.lru_cache(maxsize=None)
def system_distance(s1, s2):
    return nx.shortest_path_length(subgraph, s1, s2)

for n in subgraph:
    print(systems[n]["security_status"])


0.8897628784179688
0.9459645748138428
0.874584972858429
0.9120169878005981
0.9747262001037598
0.8236534595489502
0.7341722846031189
0.6638108491897583
0.9868146181106567
0.9599952101707458
0.9127476215362549
1.0
0.9459131360054016
0.9639235734939575
0.9531232118606567
1.0
0.7452549338340759
0.8028178811073303
0.9066284894943237
0.9467487335205078
0.6657173037528992
0.720814049243927
0.7300935983657837
0.8196477890014648
0.8712233901023865
0.8166387677192688
0.8397002816200256
0.740705132484436
0.7185566425323486
0.6860520243644714
0.8035482168197632
0.7714352011680603
0.7112933993339539
0.8409210443496704
0.7785352468490601
0.8074703812599182
0.8345176577568054
0.8308547735214233
0.7944237589836121
0.7080867290496826
0.6589382886886597
0.7411492466926575
0.6682383418083191
0.8133149147033691
0.829997718334198
0.8369776010513306
0.7791685461997986
0.6855884790420532
0.8832270503044128
0.7291611433029175
0.8472999334335327
0.670042097568512
0.7127556204795837
0.803999125957489
0.8934569

In [5]:
# Load contracts in regions
esi_op = app.op['get_universe_regions']()
region_ids = esi_client.request(esi_op).data

def get_contracts(region_id):
    esi_op = app.op['get_contracts_public_region_id'](region_id = region_id);
    return esi_client.request(esi_op).data

@functools.lru_cache(maxsize=None)
def get_station(station_id):
    esi_op = app.op['get_universe_stations_station_id'](station_id = station_id);
    return esi_client.request(esi_op).data

@functools.lru_cache(maxsize=None)
def in_highsec(station_id):
    try:
        return get_station(station_id).system_id in subgraph
    except:
        return False

contracts_region = {
    region_id : {
        c.contract_id : c
        for c in get_contracts(region_id)
        if c.type == 'courier' and in_highsec(c.start_location_id) and in_highsec(c.end_location_id)
    }
    for region_id in tqdm(region_ids)
}

contracts = {
    c.contract_id : c
    for region_id in tqdm(region_ids)
    for c in get_contracts(region_id)
    if c.type == 'courier' and in_highsec(c.start_location_id) and in_highsec(c.end_location_id)
}

@functools.lru_cache(maxsize=None)
def get_region_id(system_id):
    return constellations[systems[system_id]["constellation_id"]].region_id

In [6]:
len(contracts)

45

In [7]:
class State:
    def __init__(self, ms):
        self._system = ms.system
        self._station = ms.station
        
        self._volume_limit = ms.volume_limit
        self._collateral_limit = ms.collateral_limit
        self._time_left = ms.time_left
        
        self._accepted = frozenset(ms.accepted)
        self._loaded = frozenset(ms.loaded)
        self._finished = frozenset(ms.finished)
        
        self._value_cache = None
        self._encode_cache = None
    
    @property
    def system(self):
        return self._system
    
    @property
    def station(self):
        return self._station
    
    @property
    def volume_limit(self):
        return self._volume_limit
    
    @property
    def collateral_limit(self):
        return self._collateral_limit
    
    @property
    def time_left(self):
        return self._time_left
    
    @property
    def accepted(self):
        return self._accepted
    
    @property
    def loaded(self):
        return self._loaded
    
    @property
    def finished(self):
        return self._finished
    
    def __hash__(self):
        return (
            hash(self.system)
            ^ hash(self.station)
            ^ hash(self.time_left)
            ^ hash(self.volume_limit)
            ^ hash(self.collateral_limit)
            ^ hash(self.accepted)
            ^ hash(self.loaded)
            ^ hash(self.finished)
        )
    
    def __eq__(self, other):
        return (
            self.system == other.system
            and self.station == other.station
            and self.time_left == other.time_left
            and self.volume_limit == other.volume_limit
            and self.collateral_limit == other.collateral_limit
            and self.accepted == other.accepted
            and self.loaded == other.loaded
            and self.finished == other.finished
        )
    
    def __repr__(self):
        (vleft, cleft) = self.limits_left()
        vmax = self.volume_limit
        cmax = self.collateral_limit
        v = vmax - vleft
        c = cmax - cleft
        
        def cinfo(cid):
            def locinfo(lid):
                if(self.station == lid):
                    return "Here"
                
                system = get_station(lid).system_id
                if(system == self.system):
                    return "In system"
                
                return "{} jumps".format(nx.shortest_path_length(subgraph, system, self.system))
            
            c = contracts[cid]
                
            return "{cid} ({l1} -> {l2})".format(
                l1 = locinfo(c.start_location_id),
                l2 = locinfo(c.end_location_id),
                cid = cid
            )
        return """
State:
    Time left: {t}
    
    System:  {system}
    Station: {station}
    
    Collateral: {c} / {cmax} ({cleft} left)
    Cargo:      {v} / {vmax} ({vleft} left)
    
    Accepted:  {acc}
    Loaded:    {loa}
    Delivered: {fin}
    
    Value:     {val}
        """.format(
            t = self.time_left,
            system = self.system,
            station = self.station,
            c = c, cmax = cmax, cleft = cleft,
            v = v, vmax = vmax, vleft = vleft,
            acc = [cinfo(c) for c in self.accepted],
            loa = [cinfo(c) for c in self.loaded],
            fin = [cinfo(c) for c in self.finished],
            val = self.value()
        )
    
    def limits_left(self):
        total_vol = 0
        total_col = 0
        
        for cid in self.loaded:
            total_vol += contracts[cid]["volume"]
        
        for cid in list(self.accepted) + list(self.loaded):
            total_col += contracts[cid]["collateral"]
        
        return (self.volume_limit - total_vol, self.collateral_limit - total_col)
    
    def validate(self):
        (v, c) = self.limits_left()
        assert v >= 0, 'Volume exceeded'
        assert c >= 0, 'Collateral exceeded'
        
        assert self.time_left >= 0, 'Out of time'
        
        # Make sure we don't accept contracts we can't deliver at all
        for c in self.accepted:
            assert contracts[c].volume <= self.volume_limit, 'Accepted contract {} exceeds total volume'.format(c)
    
    def _value(self):
        accepted_factor = 1e-5
        loaded_factor = 1e-1
        
        total = 0
        
        for c in self.finished:
            total += contracts[c].reward
        
        # In principle we would be done here,
        # but training this function is hard
        #
        # To bootstrap, we add some additional
        # terms to incentivize the agent to pick
        # up contracts and move towards the target
        
        for c in self.accepted:
            con = contracts[c]
            fac = accepted_factor
            if con.start_location_id != self.station:
                dist = system_distance(self.system, get_station(con.start_location_id).system_id)
                fac /= (2 + dist)
            
            total += fac * con.reward
        
        for c in self.loaded:
            con = contracts[c]
            fac = loaded_factor
            if con.end_location_id != self.station:
                dist = system_distance(self.system, get_station(con.end_location_id).system_id)
                fac /= (2 + dist)
            
            fac += accepted_factor
            
            total += fac * con.reward
        
        return total
    
    def value(self):
        if self._value_cache is None:
            self._value_cache = self._value()
        
        return self._value_cache
    
    # Encodes the state into a NN compatible representation
    def _encode(self):        
        # Encode the state of the ship
        total_vol = 0
        total_col = 0
        
        for cid in self.loaded:
            total_vol += contracts[cid]["volume"]
        
        for cid in list(self.accepted) + list(self.loaded):
            total_col += contracts[cid]["collateral"]
        
        ship_state = [
                self.time_left,
                total_vol,
                total_col,
                self.volume_limit,
                self.collateral_limit,
                self.volume_limit - total_vol,
                self.collalteral_limit - total_vol
        ]
        
        # Encode the position
        distances = nx.shortest_path_length(subgraph, self.system)
        position_state = [
            distances[l]
            for l in landmarks
        ]
        
        # Encodes information about a contract into an array
        def encode_contract(c):
            basic_data = [
                c.collateral,
                c.volume,
                
                1 if c.contract_id in self.accepted else 0,
                1 if c.contract_id in self.loaded else 0,
                1 if c.contract_id in self.finished else 0
            ]
            
            distance_data = [
                system_distance(get_station(s).system_id, l)
                for l in [self.system] + landmarks
                for s in (c.start_location_id, c.end_location_id)
            ]
            
            return basic_data + distance_data
        
        contract_state = [
            encode_contract(c)
            for c in contracts
        ]
        
        return {
            'static' : tf.constant(
                ship_state + position_state,
                dtype = np.float32
            ),
            
            'contracts' : tf.constant(
                contract_state,
                dtype = np.float32
            )
        }
    
    def encode(self):
        if self._encode_cache is None:
            self._encode_cache = self.encode()
        
        return self._encode_cache
        

# Helper class for incremental state modification
class MutableState:
    def __init__(self, other = None):
        if other is None:
            # Position of the ship
            self.system = None  # Current system
            self.station = None # Current no. of station in system
            
            self.volume_limit = 0
            self.collateral_limit = 0
            
            self.time_left = 0

            # Contracts
            self.accepted = set([])
            self.loaded = set([])
            self.finished = set([])
        else:
            self.system = other.system
            self.station = other.station
            
            self.time_left = other.time_left
            
            self.volume_limit = other.volume_limit
            self.collateral_limit = other.collateral_limit
            
            self.accepted = set(other.accepted)
            self.loaded = set(other.loaded)
            self.finished = set(other.finished)

In [8]:
# Action calculus on states

warp_time = 1.0
accept_range = 5

max_volume = 1e6

# Wrapper that allows an action to just modify a mutable state representation
# and performs validity checking of resulting state
def action(f):
    def apply(state):
        s = MutableState(state)
        f(s)
        s = State(s)
        
        s.validate()
        
        return s
    
    return apply

def warp_to_station(station_id):
    station = get_station(station_id)
    
    @action
    def apply(s):
        assert s.system == station.system_id, 'Station {} not in system {}'.format(station_id, s.system)
        assert s.station != station_id, 'Cannot warp from to same station'
        
        s.station = station_id
        s.time_left -= warp_time
    
    return apply

def jump_to_system(system_id):
    system = systems[system_id]
    
    @action
    def apply(s):
        assert subgraph.has_edge(s.system, system_id), 'System {} not adjacent to current system {}'.format(system_id, s.system)
        
        s.system = system_id
        s.station = None
        s.time_left -= warp_time
    
    return apply

@functools.lru_cache(maxsize=None)
def warp_to(station_id):
    station = get_station(station_id)
    
    distances = nx.shortest_path_length(subgraph, station.system_id)
    
    @action
    def do_warp_to(s):
        assert s.station != station_id, 'Cannot warp from to same station'
        
        if s.system == station.system_id:
            t = warp_time
        else:
            #t = warp_time * nx.shortest_path_length(subgraph, s.system, station.system_id)
            t = warp_time * distances[s.system]
        
        s.station = station_id
        s.system = station.system_id
        
        s.time_left -= t
    
    return do_warp_to
        

def accept_contract(contract):
    @action
    def do_accept(s):
        assert contract.contract_id not in s.accepted, 'Contract already accepted'
        assert contract.contract_id not in s.loaded, 'Contract already loaded'
        assert contract.contract_id not in s.finished, 'Contract already finished'
        
        s.accepted.add(contract.contract_id)
    
    return do_accept

def load_contract(contract):
    @action
    def do_load(s):
        assert s.station == contract.start_location_id, 'Not in pickup station'
        assert contract.contract_id in s.accepted, 'Contract not accepted'
        
        s.accepted.remove(contract.contract_id)
        s.loaded.add(contract.contract_id)
        
    return do_load

def finish_contract(contract):
    @action
    def do_finish(s):
        assert s.station == contract.end_location_id, 'Not in delivery station'
        assert contract.contract_id in s.loaded, 'Contract not loaded'
        
        s.loaded.remove(contract.contract_id)
        s.finished.add(contract.contract_id)
    
    return do_finish

def actions(s):
    (vol, col) = s.limits_left()
    
    accept_actions = [
        accept_contract(c)
        for c in contracts.values()
        if c.volume <= vol and c.collateral <= col
    ]
    
    #jump_actions = [
    #    jump_to_system(sys)
    #    for sys in subgraph.neighbors(s.system)
    #]
    
    #dock_actions = [
    #    warp_to_station(sta)
    #    for sta in systems[s.system].get("stations", [])
    #]
    
    warp_targets = [
        contracts[cid].start_location_id
        for cid in s.accepted
    ] + [
        contracts[cid].end_location_id
        for cid in s.loaded
    ]
    
    warp_actions = [
        warp_to(t)
        for t in warp_targets
    ]
    
    load_actions = [
        load_contract(contracts[cid])
        for cid in s.accepted
        if s.station is not None and contracts[cid].start_location_id == s.station
    ]
    
    finish_actions = [
        finish_contract(contracts[cid])
        for cid in s.loaded
        if s.station is not None and contracts[cid].end_location_id == s.station
    ]
    
    result = warp_actions + load_actions + finish_actions + accept_actions
    
    return result

def apply_action(a, s):
    try:
        return a(s)
    except AssertionError as e:
        return None

def apply_actions(s):
    return [
        s2
        for s2 in [apply_action(a, s) for a in actions(s)]
        if s2 is not None
    ]
    

In [9]:
# Build a node into a state space traversal graph by expanding every node that maximizes a given heuristic
def expand(root, heuristic, count, sort_every = 5):
    result = nx.DiGraph()
    result.add_node(root)
    
    unexpanded = [root]
    
    for counter in trange(0, count):
        if not unexpanded:
            break;
        s = unexpanded.pop()
        
        new_states = apply_actions(s)
        
        for s2 in new_states:
            if s2 not in result:
                result.add_node(s2)
                #unexpanded.append(s2)
                unexpanded.insert(0, s2)
            
            result.add_edge(s, s2)
        
        if counter % sort_every == 0:
            unexpanded.sort(key = heuristic)
        
        counter += 1

    return result, frozenset(unexpanded)

def get_values(root, g, unexpanded, heuristic):
    values = {}
    for node in nx.dfs_postorder_nodes(g, root):
        # Unexpanded nodes are weighted with their heuristic
        if node in unexpanded:
            values[node] = heuristic(node)
            continue
        
        # Expanded nodes are weighted with the maximum of successor nodes
        # and the known value of the node
        val = node.value()
        for n2 in g.successors(node):
            if n2 not in values:
                continue
                
            val = max(val, values[n2])
        
        values[node] = val
    
    return values

In [11]:
s = MutableState()

s.system = root_id
s.time_left = 40.0

ship = 'tayra'

if ship == 'charon':
    s.volume_limit = 1350000
    s.collateral_limit = 750000000
elif ship == 'tayra':
    s.volume_limit = 8395
    s.collateral_limit = 100000000

expanded = set([])

def heur(x):
    return x.value()

g, unexp = expand(State(s), heur, 30000)

heurmax = max(*[heur(n) for n in g])
print(heurmax)

160000.0


In [ ]:
import cProfile

cProfile.run("expand(State(s), heur, 10000)")

In [ ]:
cycles = nx.simple_cycles(g);

for c in cycles:
    print(c)

In [ ]:
plt.figure(figsize=(20, 20))

nx.draw_networkx(
    g,
    labels = {n:systems[n.system]["name"] for n in g},
    node_size = [300 * heur(n) / heurmax for n in g],
    node_color = [heur(n) / heurmax for n in g],
    cmap = 'jet'
)

c = matplotlib.colorbar.ColorbarBase(plt.gca(), cmap='jet')

In [ ]:
vals = get_values(State(s), g, unexp, heur)

In [ ]:
len(g)

In [ ]:
plt.figure(figsize=(20, 20))

valmax = max(*vals.values())

subg = g.subgraph([n for n in g if vals[n] == valmax])

nx.draw_networkx(
    subg,
    labels = {n:systems[n.system]["name"] for n in subg},
    node_size = [30 * n.time_left for n in subg],
    node_color = [300 * heur(n) / heurmax for n in subg],
    alpha = 1.0,
    cmap = 'jet'
)

plt.show()

for n in nx.dfs_preorder_nodes(g, State(s)):
    if vals[n] < valmax:
        continue
    
    print(n)

In [ ]:
for n in nx.dfs_preorder_nodes(g, State(s)):
    if not n.loaded:
        continue
    
    print(n)

In [ ]:
plt.hist([heur(n) for n in g]);

In [ ]:
vals

In [ ]:
systems[root_id]